### Neutral reviews (3 stars)

In [1]:
%%capture
!pip install -r ../requirements.txt

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata
import fasttext
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.cluster.util import cosine_distance
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from wordcloud import WordCloud
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import model_selection, svm
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
df_raw = pd.read_csv('../data/raw/Big_AHR.csv')

In [4]:
df_3_stars = df_raw[df_raw['rating']==3]

In [15]:
for text in df_3_stars['review_text'].sample(5).tolist():
    print(text)
    print()

Está bien situado, aunque las calles de alrededor son muy sucias...por la noche se oía ruido de la calle. Tuvimos bichos en la habitación, aunque el personal fumigó y nos ofreció el cambio la situación fue algo desagradable. Aun así, tiene buen precio y buen desayuno (merece la pena cogerlo).

La ubicación es perfecta, entra el acceso a la Alcazaba y la Catedral. Puedes recorrer todo el centro e ir a la playa de la Malagueta y la zona del MuelleUno andando. El parking está a escasos metros y el acceso al mismo y las plazas son correctos. La habitación daba a un patio interior y era tranquila. El hotel está algo antiguo, pero cumple para una estancia de ciudad. Alrededor hay muchos establecimientos para desayunar, comer y cenar.

La posada Doña Lupe se puede ver de muchas maneras. No es un lugar para estarse en las habitaciones, ya que éstas sólo sirven para dormir y gracias. Está poco cuidado (faltaban muchas bombetas, por ejemplo) y se podría decir que la limpieza era la justa. Aún as

In this sample of neutral reviews, it can be seen that the comments highlight both positive and negative aspects.
Therefore, each comment will be separated into sentences, with the aim of being classified as positive or negative.

The division of each comment into sentences and the text processing was done *using create_df_sentences.py*

In [5]:
df_sentences_3_stars = pd.read_csv('../data/interim/sentences_3_stars.csv')

df_sentences_1_star = pd.read_csv('../data/interim/sentences_1_star.csv')
df_sentences_5_stars = pd.read_csv('../data/interim/sentences_5_stars.csv')

In [6]:
len(df_sentences_1_star)

11885

In [7]:
len(df_sentences_5_stars)

53205

In [8]:
df_sentences_5_stars = df_sentences_5_stars.sample(len(df_sentences_1_star))

In [9]:
print('Examples of negative sentences:')
df_sentences_1_star.sample(3)

Examples of negative sentences:


,text,review_index,text_processed
6117,Dejadez,928,dejadez
11294,Voy a hacer el checkout sin poder desayunar,1601,voy hacer checkout sin poder desayunar
2029,Vale,321,vale


In [30]:
print('Examples of positive sentences:')
df_sentences_5_stars.sample(3)

Examples of positive sentences:


,text,text_processed
26391,Una decoración muy chula y el servicio y atenc...,decoracion chula servicio atencion impecables
23917,Habitación confortable para su precio,habitacion confortable precio
47929,Totalmente recomendable!!,totalmente recomendable


In [10]:
# Add column to indicate sentiment
df_sentences_1_star['label'] = 'Negative'
df_sentences_5_stars['label'] = 'Positive'

In [11]:
# Combine positive and negative sentences into one df
df_sentences_labeled = pd.concat([df_sentences_1_star, df_sentences_5_stars]).reset_index(drop=True)

In [12]:
df_sentences_labeled['label'] = df_sentences_labeled['label'].astype('category')

In [13]:
df_sentences_labeled

,text,review_index,text_processed,label
0,Suciedad intolerable,0,suciedad intolerable,Negative
1,Jamás me he encontrado un nivel de suciedad co...,0,jamas encontrado nivel suciedad,Negative
2,Ni en hostales de una estrella,0,hostales estrella,Negative
3,Cuando llegamos tuvimos que solicitar el cambi...,0,llegamos solicitar cambio habitacion ver misma...,Negative
4,Durante el proceso de cambio las enseñé en rec...,0,proceso cambio ensene recepcion miraron cara,Negative
...,...,...,...,...
23765,El evento superó todas las expectativas que te...,375,evento supero todas expectativas no solo sino ...,Positive
23766,"5 km a pie del centro principal, donde hay un ...",2421,5 km pie centro principal monton bares restaur...,Positive
23767,En verano volvemos! Enhorabuena y gracias por...,7477,verano volvemos enhorabuena gracias ofrecer lu...,Positive
23768,Granada bonita,3665,granada bonita,Positive


In [22]:
df_sentences_labeled[df_sentences_labeled.text_processed.isna()]

,text,review_index,text_processed,label
486,…,80,NaN,Negative
493,…,81,NaN,Negative
508,…,83,NaN,Negative
524,…,85,NaN,Negative
531,…,86,NaN,Negative
...,...,...,...,...
22426,…,4370,NaN,Positive
22572,…,6378,NaN,Positive
23085,"Отличный хозяин !Общительный, добродушный",6149,NaN,Positive
23142,…,5006,NaN,Positive


In [23]:
df_clean = df_sentences_labeled[df_sentences_labeled.text_processed.notna()]

##### Construction of the model for sentences classification

In [24]:
X = df_clean['text_processed']
y = df_clean['label']

In [25]:
X.shape

(23577,)

In [26]:
type(X)

pandas.core.series.Series

In [27]:
y.shape

(23577,)

In [28]:
type(y)

pandas.core.series.Series

In [29]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, random_state=12)

In [30]:
# Now lets create our model
classifier = Pipeline([
    ('count_vectorizer', CountVectorizer(max_features=200)),
    ('tfidf', TfidfTransformer()),
    ('model', MultinomialNB())
])

In [17]:
model = Pipeline([
    ('count_vectorizer', CountVectorizer(max_features=200, ngram_range=(1,2), lowercase=False)),
    ('tfidf_transformer', TfidfTransformer()),
    ('classifier', MultinomialNB())     # SVC(kernel='rbf', probability=True)
])

In [31]:
classifier.fit(X_train, y_train)

Pipeline(steps=[('count_vectorizer', CountVectorizer(max_features=200)),
                ('tfidf', TfidfTransformer()), ('model', MultinomialNB())])

In [32]:
y_pred = classifier.predict(X_val)

In [34]:
classifier.score(X_train, y_train)

0.7395091053048297

In [35]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

    Negative       0.76      0.65      0.70      2935
    Positive       0.70      0.79      0.74      2960

    accuracy                           0.72      5895
   macro avg       0.73      0.72      0.72      5895
weighted avg       0.73      0.72      0.72      5895



In [36]:
df_neutral = df_sentences_3_stars[df_sentences_3_stars.text_processed.notna()]

In [37]:
df_neutral

,text,review_index,text_processed
0,No es oro todo lo que reluce,0,no oro reluce
1,"El hotel en general está bien, las habtiacione...",0,general bien habtiaciones espaciosas personal ...
2,Pero tiene dos grandes fallos: El primero es q...,0,dos grandes fallos primero wifi no llegaba bie...
3,El segundo fallo es que no se les ocurre otra ...,0,segundo fallo no ocurre cosa poner edredon nor...
4,Un buen hotel con mucho ruido,1,buen ruido
...,...,...,...
14856,Pues fuimos a este hostal porque vimos en inte...,2272,pues hostal vimos internet bien dieron habitac...
14857,"El desayuno admisible y tenían parking, lejos ...",2272,desayuno admisible parking lejos menos podias ...
14858,aceptable,2273,aceptable
14859,"No es un mar de lujos, pero está bien situado ...",2273,no mar lujos bien situado limpio


In [38]:
X_neutral = df_neutral['text_processed']

In [39]:
y_neutral = classifier.predict(X_neutral)

In [40]:
y_neutral

array(['Negative', 'Positive', 'Negative', ..., 'Positive', 'Positive',
       'Negative'], dtype='<U8')

In [41]:
df_n = pd.DataFrame({'label':y_neutral})

In [42]:
df_n.label.value_counts()

Positive    8171
Negative    6571
Name: label, dtype: int64

In [ ]:
# Hacer test chi cuadrado
# Probar con 2 y 4 estrellas